In [4]:
all_classes = {"test1", "test2", "test3"}
vehicle_classes = {"bus", "car", "motorcycle", "truck"}
accident_classes = set(all_classes) - vehicle_classes

In [5]:
print("All classes:", all_classes)
print("Vehicle classes:", vehicle_classes)
print("Accident classes:", accident_classes)

All classes: {'test1', 'test2', 'test3'}
Vehicle classes: {'bus', 'car', 'motorcycle', 'truck'}
Accident classes: {'test1', 'test2', 'test3'}


In [ ]:
import cv2
import streamlit as st
import time

def main():
    st.title("Webcam Live Feed")
    run = st.checkbox('Run')
    FRAME_WINDOW = st.image([])
    camera = cv2.VideoCapture(1)
    print(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"cv2.CAP_PROP_FPS: {camera.get(cv2.CAP_PROP_FPS)}")

    while run:
        _, frame = camera.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        FRAME_WINDOW.image(frame)

        frame_count = 0
        start_time = time.time()

        while frame_count < 100:
            ret, frame = camera.read()
            if not ret:
                break
            frame_count += 1

        elapsed_time = time.time() - start_time
        actual_fps = frame_count / elapsed_time
        print(f"Actual FPS: {actual_fps}")

    else:
        st.write('Stopped')

In [ ]:
_, frame = camera.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame_count = 0
start_time = t.time()
while frame_count < 100:
    ret, frame = camera.read()
    if not ret:
        break
    frame_count += 1
elapsed_time = t.time() - start_time
actual_fps = frame_count / elapsed_time

# Dapatkan FPS dari webcam FPS = camera.get(cv2.CAP_PROP_FPS) #5
FPS = actual_fps  # Default FPS 
print(f"FPS: {FPS}")
print(f"camera.get(cv2.CAP_PROP_FPS): {camera.get(cv2.CAP_PROP_FPS)}")

In [1]:
import numpy as np

def convert_coordinates(coords, old_res, new_res):
    """
    Mengonversi koordinat dari resolusi asli ke resolusi target.

    Args:
        coords (np.ndarray): Array koordinat dengan bentuk (N, 2).
        old_res (tuple): Resolusi asli (width, height).
        new_res (tuple): Resolusi target (width, height).

    Returns:
        np.ndarray: Koordinat baru pada resolusi target.
    """
    old_width, old_height = old_res
    new_width, new_height = new_res

    scale_x = new_width / old_width
    scale_y = new_height / old_height

    # Skalakan koordinat
    converted_coords = coords * np.array([scale_x, scale_y])
    return converted_coords.astype(int)


# Koordinat asli
source_coords = np.array([
    [290,197],
    [516,211],
    [484,358],
    [100,333]
])

# Resolusi asli dan target
old_resolution = (640, 360)
new_resolution = (1280, 720)

# Konversi koordinat
converted_coords = convert_coordinates(source_coords, old_resolution, new_resolution)
print("Koordinat baru:", converted_coords)


Koordinat baru: [[ 580  394]
 [1032  422]
 [ 968  716]
 [ 200  666]]


In [ ]:
def scale_coordinates(coords, old_res, new_res):
    scale_x = new_res[0] / old_res[0]
    scale_y = new_res[1] / old_res[1]
    
    return [(int(x * scale_x), int(y * scale_y)) for x, y in coords]

def format_coordinates(coords):
    return ';'.join([f"{x},{y}" for x, y in coords])

# Koordinat asli
coordinates = [(930, 253), (1219, 233), (1635, 1125), (946, 1172)]

# Resolusi
old_resolution = (1920, 1080)  # 1080p
new_resolution = (1280, 720)   # 720p

# Konversi
coordinates_360p = scale_coordinates(coordinates, old_resolution, new_resolution)
formatted_coordinates = format_coordinates(coordinates_360p)
print(formatted_coordinates)

620,168;812,155;1090,750;630,781


Model Architecture

In [ ]:
from torchinfo import summary
import torch
from ultralytics import YOLO               

# Load the YOLO model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo_model = YOLO('l-vehicle.pt')  # Load the YOLO model
core_model = yolo_model.model.to(device)   # Access the core PyTorch model

# Input size (e.g., 3 channels, 640x640 for YOLO)
summary(core_model)  # Note: Batch size is included as 1


Layer (type:depth-idx)                                       Param #
DetectionModel                                               --
├─Sequential: 1-1                                            --
│    └─Conv: 2-1                                             --
│    │    └─Conv2d: 3-1                                      (1,728)
│    │    └─BatchNorm2d: 3-2                                 (128)
│    │    └─SiLU: 3-3                                        --
│    └─Conv: 2-2                                             --
│    │    └─Conv2d: 3-4                                      (73,728)
│    │    └─BatchNorm2d: 3-5                                 (256)
│    │    └─SiLU: 3-6                                        --
│    └─C3k2: 2-3                                             --
│    │    └─Conv: 3-7                                        (16,640)
│    │    └─Conv: 3-8                                        (66,048)
│    │    └─ModuleList: 3-9                                  (91,136)


In [ ]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO('l-vehicle-accident.pt')

# Print the architecture
print(model.model)


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

In [ ]:
# Print detailed architecture layer by layer
for name, layer in model.model.named_modules():
    print(f"{name}: {layer}")


: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
 

In [ ]:
import torch
from ultralytics import YOLO

model = YOLO('l-vehicle-accident.pt').model  # Core PyTorch model
dummy_input = torch.randn(1, 3, 640, 640)  # Dummy input, hanya untuk ekspor
torch.onnx.export(model, dummy_input, "yolo_model.onnx", opset_version=11)
